In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
from init_model import Transformer as PreProcessor
from gen_model import Transformer as Generator
from text_utils import TextEncoder, encode_text
import tensorflow.keras as keras

In [0]:
ENCODER_PATH = 'model/encoder_bpe_40000.json'
BPE_PATH = 'model/vocab_40000.bpe'
encoder = TextEncoder(ENCODER_PATH, BPE_PATH)
decoder = encoder.decoder

In [0]:
n_vocab = 40478

In [0]:
sess =  tf.Session()
keras.backend.set_session(sess)
preprocessor = PreProcessor("preprocessor", n_vocab)
generator = Generator("generator", n_vocab)

In [9]:
X = tf.placeholder(tf.int32, [1, 512, 2])
L = tf.placeholder(tf.int32, [])
token, mem_k, mem_v = preprocessor([L, X])

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
T = tf.placeholder(tf.int32, [1, 1, 2])
K = tf.placeholder(tf.float32, [12, 1, 12, 64, 512])
V = tf.placeholder(tf.float32, [12, 1, 12, 512, 64])
I = tf.placeholder(tf.int32, [])
next_token, next_mem_k, next_mem_v = generator([T, K, V, I])

In [11]:
generator.load_weights('model/model.ckpt')
preprocessor.load_weights('model/model.ckpt')

In [0]:
txt = [
    "Daniel is approached by a young man calling himself Paul Sunday, who tells him of an oil deposit under his family's property in Little Boston, California. Daniel attempts to buy the farm at a bargain price. However, Paul's apparent twin brother, Eli, demands $10,000 for the church, of which he is pastor. An agreement is made for $5,000 after the well begins producing, and Daniel acquires all the available surrounding land, except for one holdout: William Bandy. Oil production begins, but after an accident kills a worker and a gas blowout deafens H. W., Eli blames the disasters on the well not being properly blessed. When Eli"
]

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
init_len, x = encode_text(txt)
init_len

137

In [0]:
gen_len = 200

In [0]:
t, k, v = sess.run([token, mem_k, mem_v], {X: x, L: init_len - 1})

In [0]:
x[0, init_len, 0] = t
gen_len -= 1

In [0]:
gen_tokens = [decoder[t].replace("</w>", "")]

In [0]:
for step in range(gen_len):
    prev_token = x[:, init_len + step : init_len + step + 1, :]
    t, k, v = sess.run([next_token, next_mem_k, next_mem_v], {T: prev_token, K: k, V: v, I: init_len + step})
    gen_tokens.append(decoder[t].replace("</w>", ""))
    x[0, init_len + step + 1, 0] = t

In [23]:
' '.join(gen_tokens)

'asks him why he is so angry , he says : " i do n\'t want to be blamed for the fact that i \'m not a man to be blamed for . " he paused , and looked at daniel . " i \'m sorry , mr. ban dy . i \'m sorry i \'ve been so rude . " he bowed his head and said , " i \'m sorry . " he went on , " and daniel was not sure what to say . " i \'m sorry , " he said , " i \'m a little upset . i \'m just a little upset . " he was not sure what to do . " i \'m sorry , " he said , " i \'m not sure what i \'m doing here . i \'m just a friend of the family . " he looked at daniel , who was still standing there , and said , " i \'m sorry . i \'m just a little upset . i \'m not sure i \'m not sure i \'m going to be able to get over it . " he was not sure'